snake

In [ ]:
import pygame
import random
import time

# Initialize Pygame
pygame.init()

# Set up the screen
WIDTH, HEIGHT = 600, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")

# Set colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Constants
GRIDSIZE = 20
UP = (0, -1)
DOWN = (0, 1)
LEFT = (-1, 0)
RIGHT = (1, 0)

# Snake class
class Snake:
    def __init__(self):
        self.length = 1
        self.positions = [((WIDTH / 2), (HEIGHT / 2))]
        self.direction = random.choice([UP, DOWN, LEFT, RIGHT])
        self.color = GREEN
        self.score = 0
        self.level = 1
        self.speed = 5

    def get_head_position(self):
        return self.positions[0]

    def turn(self, point):
        if self.length > 1 and (point[0] * -1, point[1] * -1) == self.direction:
            return
        else:
            self.direction = point

    def move(self):
        cur = self.get_head_position()
        x, y = self.direction
        new = (((cur[0] + (x*self.speed)) % WIDTH), (cur[1] + (y*self.speed)) % HEIGHT)
        if len(self.positions) > 2 and new in self.positions[2:]:
            self.reset()
        else:
            self.positions.insert(0, new)
            if len(self.positions) > self.length:
                self.positions.pop()

    def reset(self):
        self.length = 1
        self.positions = [((WIDTH / 2), (HEIGHT / 2))]
        self.direction = random.choice([UP, DOWN, LEFT, RIGHT])
        self.score = 0
        self.level = 1
        self.speed = 5

    def draw(self, surface):
        for p in self.positions:
            r = pygame.Rect((p[0], p[1]), (GRIDSIZE, GRIDSIZE))
            pygame.draw.rect(surface, self.color, r)
            pygame.draw.rect(surface, BLACK, r, 1)

    def handle_keys(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    self.turn(UP)
                elif event.key == pygame.K_DOWN:
                    self.turn(DOWN)
                elif event.key == pygame.K_LEFT:
                    self.turn(LEFT)
                elif event.key == pygame.K_RIGHT:
                    self.turn(RIGHT)

# Food class
class Food:
    def __init__(self, x, y, color, timer):
        self.position = (x, y)
        self.color = color
        self.timer = timer
        self.spawn_time = time.time()

    def draw(self, surface):
        r = pygame.Rect((self.position[0], self.position[1]), (GRIDSIZE, GRIDSIZE))
        pygame.draw.rect(surface, self.color, r)

    def is_expired(self):
        return time.time() - self.spawn_time >= self.timer

# Function to generate random food with different colors and timers
def generate_food(snake):
    while True:
        x = random.randint(0, (WIDTH - GRIDSIZE) // GRIDSIZE) * GRIDSIZE
        y = random.randint(0, (HEIGHT - GRIDSIZE) // GRIDSIZE) * GRIDSIZE
        if (x, y) not in snake.positions:
            color = random.choice([RED, BLUE])  # Randomly choose between red and blue food
            timer = random.randint(5, 10)   # Assign random timer (5 to 10 seconds)
            return Food(x, y, color, timer)

# Function to check for collision with food
def food_collision(snake, food):
    return snake.get_head_position() == food.position

# Function to draw the level and score
def draw_info(surface, font, snake):
    level_text = font.render(f"Level: {snake.level}", True, BLACK)
    score_text = font.render(f"Score: {snake.score}", True, BLACK)
    surface.blit(level_text, (10, 10))
    surface.blit(score_text, (10, 30))

# Main function
def main():
    clock = pygame.time.Clock()
    snake = Snake()
    food_list = []
    font = pygame.font.Font(None, 24)

    # Timer for food disappearance
    food_timer = time.time()

    while True:
        screen.fill(WHITE)

        # Check for events
        snake.handle_keys()
        snake.move()

        # Check for border collision
        if snake.get_head_position()[0] < 0 or snake.get_head_position()[0] >= WIDTH or \
                snake.get_head_position()[1] < 0 or snake.get_head_position()[1] >= HEIGHT:
            snake.reset()

        # Check for food collision
        for food in food_list:
            if food_collision(snake, food):
                snake.score += 1
                snake.length+=1
                food_list.remove(food)

        # Generate new food if timer elapsed
        if time.time() - food_timer >= 1:
            food_list.append(generate_food(snake))
            food_timer = time.time()

        # Remove expired foods
        food_list = [food for food in food_list if not food.is_expired()]

        # Check for level up
        if snake.score >= snake.level * 3:
            snake.length+=1
            snake.level += 1
            snake.speed += 1

        # Draw everything
        snake.draw(screen)
        for food in food_list:
            food.draw(screen)
        draw_info(screen, font, snake)

        pygame.display.flip()
        clock.tick(snake.speed)

# Execute main function
if __name__ == "__main__":
    main()

# Quit Pygame
pygame.quit()

racer

In [ ]:
import pygame
import random
import time
import sys

pygame.init()

fps = 60
time1 = pygame.time.Clock()
font = pygame.font.SysFont("Verdana",60)
font_small = pygame.font.SysFont("Verdana", 20)
game_over = font.render("Game Over", True, (0, 0, 0))
bg = pygame.image.load("images/background.png")
SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 1
SCORE = 0
SCORE2 = 0 
DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Race")
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("images/Enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        global SCORE
        self.rect.move_ip(0,SPEED*5)
        if self.rect.top > SCREEN_HEIGHT:
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)


class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("images/Player.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, SCREEN_HEIGHT - 80)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0:
            if pressed_keys[pygame.K_LEFT]:
                self.rect.move_ip(-10, 0)
        if self.rect.right < SCREEN_WIDTH:
            if pressed_keys[pygame.K_RIGHT]:
                self.rect.move_ip(10, 0)


class Coins(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("images/coin.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

    def move(self):
        global SCORE2
        self.rect.move_ip(0, 5)
        if self.rect.top > SCREEN_HEIGHT:
            self.rect.top = 0
            self.rect.center = (random.randint(40, SCREEN_WIDTH - 40), 0)

P1 = Player()
E1 = Enemy()
C1 = Coins()
enemies = pygame.sprite.Group()
enemies.add(E1)
all_sprites = pygame.sprite.Group()
all_sprites.add(P1)
all_sprites.add(E1)
all_sprites.add(C1)
INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 5000) 
while True:
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.5
            new_coin = Coins()  
            all_sprites.add(new_coin)
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    DISPLAYSURF.blit(bg, (0, 0))
    scores = font_small.render("Score: " + str(SCORE), True, (0, 0, 0))
    coin_score = font_small.render("Coins: " + str(SCORE2), True, (0, 0, 0))
    DISPLAYSURF.blit(scores, (10, 10))
    DISPLAYSURF.blit(coin_score, (SCREEN_WIDTH - 200, 10))
    for entity in all_sprites:
        if entity != P1:
            DISPLAYSURF.blit(entity.image, entity.rect)
            entity.move()
    DISPLAYSURF.blit(P1.image, P1.rect)
    P1.move()
    if pygame.sprite.spritecollideany(P1, enemies):
        pygame.mixer.Sound("songs/racer_images_crash.wav").play()
        time.sleep(0.5)
        DISPLAYSURF.blit(game_over, (30, 250))
        pygame.display.update()
        for entity in all_sprites:
            entity.kill()
        time.sleep(1)
        pygame.quit()
        sys.exit()
    collisions = pygame.sprite.spritecollide(P1, all_sprites, True)
    print(SCORE2)
    for coin in collisions: 
        SCORE2 += 1
    pygame.display.update()
    time1.tick(fps)

paint

In [ ]:
import pygame
import math
pygame.init() 

painting = []
icon=pygame.image.load('images/paq.png')
pygame.display.set_icon(icon)

timer = pygame.time.Clock() # We need it for future use with fps

fps = 60 # Set Frames per second

activeColor = (0, 0, 0)
activeShape = 0

w = 800 # Set Window Size
h = 600

screen = pygame.display.set_mode([w, h]) # Set Screen

pygame.display.set_caption("Paint") # Set Window Title

def drawDisplay():
    pygame.draw.rect(screen, 'gray', [0, 0, w, 100]) # Draw Display
    pygame.draw.line(screen, 'black', [0, 100], [w, 100]) # Draw Line separator
    rect = [pygame.draw.rect(screen, 'black', [10, 10, 80, 80]), 0]
    pygame.draw.rect(screen, 'white', [20, 20, 60, 60])
    circ = [pygame.draw.rect(screen, 'black', [100, 10, 80, 80]), 1]
    pygame.draw.circle(screen, 'white', [140, 50], 30)
    blue = [pygame.draw.rect(screen, (0, 0, 255), [w - 35, 10, 25, 25]), (0, 0, 255)] # Draw colors
    red = [pygame.draw.rect(screen, (255, 0, 0), [w - 35, 35, 25, 25]), (255, 0, 0)] # Draw colors
    green = [pygame.draw.rect(screen, (0, 255, 0), [w - 60, 10, 25, 25]), (0, 255, 0)] # Draw colors
    yellow = [pygame.draw.rect(screen, (255, 255, 0), [w - 60, 35, 25, 25]), (255, 255, 0)] # Draw colors
    eraser = [pygame.draw.rect(screen, (255, 255, 255), [w - 150, 20, 25, 25]), (255, 255, 255)] # Draw Eraser
    return [blue, red, green, yellow,  eraser], [rect, circ]

def drawPaint(paints):
    for paint in paints:
        if paint[2] == 1:
            pygame.draw.circle(screen, paint[0], paint[1], 15) # Draw Paint
        elif paint[2] == 0:
            pygame.draw.rect(screen, paint[0], [paint[1][0]-15, paint[1][1]-15, 30, 30]) # Draw Paint
def draw():
    global activeColor, activeShape, mouse
    if mouse[1] > 100:
        if activeShape == 0:
            pygame.draw.rect(screen, activeColor, [mouse[0]-15, mouse[1]-15, 30, 30]) # Draw
        if activeShape == 1:
            pygame.draw.circle(screen, activeColor, mouse, 15)
def draw_square(x, y, side_length):
    pygame.draw.rect(screen, (0,0,0), (x, y, side_length, side_length), 2)

# Function to draw a right triangle
def draw_right_triangle(x, y, base, height):
    points = [(x, y), (x, y + height), (x + base, y + height)]
    pygame.draw.polygon(screen,(0,0,0) , points, 2)

# Function to draw an equilateral triangle
def draw_equilateral_triangle(x, y, side_length):
    height = math.sqrt(3) * side_length / 2
    points = [(x + side_length / 2, y), (x, y + height), (x + side_length, y + height)]
    pygame.draw.polygon(screen, (0,0,0), points, 2)

# Function to draw a rhombus
def draw_rhombus(x, y, width, height):
    points = [(x + width / 2, y), (x + width, y + height / 2), (x + width / 2, y + height), (x, y + height / 2)]
    pygame.draw.polygon(screen, (0,0,0), points, 2)
run = True
while run:
    timer.tick(fps) # Set FPS
    screen.fill('white') # Fill Screen
    colors, shape = drawDisplay() # Draw Display

    mouse = pygame.mouse.get_pos() # Get Mouse Position
    draw()
    
    click = pygame.mouse.get_pressed()[0] # Get Mouse Button Pressed
    if click and mouse[1] > 100:
        painting.append((activeColor, mouse, activeShape)) # Add Mouse Position to List
    drawPaint(painting)
    draw_square(100, 100, 100)
    draw_right_triangle(300, 100, 100, 100)
    draw_equilateral_triangle(500, 100, 100)
    draw_rhombus(100, 300, 100, 100)
    for event in pygame.event.get(): # Set quit event
        if event.type == pygame.QUIT:
            run = False

        if event.type == pygame.KEYDOWN: # Set quit event
            if event.key == pygame.K_ESCAPE:
                run = False

        if event.type == pygame.KEYDOWN: # Set quit event
            if event.key == pygame.K_SPACE:
                painting = []

        if event.type == pygame.MOUSEBUTTONDOWN:
            for i in colors:
                if i[0].collidepoint(event.pos):
                    activeColor = i[1]
            for i in shape:
                if i[0].collidepoint(event.pos):
                    activeShape = i[1]
    

    pygame.display.flip() # Update Screen

ackanoid

In [ ]:
import pygame, random
'''
shortly in this game you can pass to esc and make the game pause
'''
pygame.init()
W, H = 1200, 800
is_paused = False
in_settings = False
FPS = 60
speed_increase_interval = 10000
last_speed_increase = pygame.time.get_ticks()
screen = pygame.display.set_mode((W, H), pygame.RESIZABLE)
clock = pygame.time.Clock()
done = False
bg = (0, 0, 0)
paddleW = 150
paddleH = 25
paddleSpeed = 20
paddle = pygame.Rect(W // 2 - paddleW // 2, H - paddleH - 30, paddleW, paddleH)
ballRadius = 20
ballSpeed = 6
ball_rect = int(ballRadius * 2 ** 0.5)
ball = pygame.Rect(random.randrange(ball_rect, W - ball_rect), H // 2, ball_rect, ball_rect)
dx, dy = 1, -1
game_score = 0
game_score_fonts = pygame.font.SysFont('comicsansms', 40)
game_score_text = game_score_fonts.render(f'Your game score is: {game_score}', True, (0, 0, 0))
game_pause_fonts = pygame.font.SysFont('comicsansms', 40)
game_pause_text = game_pause_fonts.render(f'Game paused', True, (187,150,143))
game_pause_rect = game_pause_text.get_rect()
game_pause_rect.center = (W // 2, H // 2)
game_score_rect = game_score_text.get_rect()
game_score_rect.center = (210, 20)
collision_sound = pygame.mixer.Sound('songs/ackanoid_sounds_catch.mp3')
block_list = [pygame.Rect(10 + 120 * i, 50 + 70 * j, 100, 50) for i in range(10) for j in range(4)]
color_list = [(random.randrange(0, 255), random.randrange(0, 255), random.randrange(0, 255)) for i in range(10) for j in range(4)]
bonus_block_index = random.randrange(len(block_list))
unbreakable_block_list = [pygame.Rect(10 + 120 * i, 120, 100, 50) for i in range(5)]
unbreakable_color = (100, 100, 100)
losefont = pygame.font.SysFont('comicsansms', 40)
losetext = losefont.render('Game Over', True, (255, 255, 255))
losetextRect = losetext.get_rect()
losetextRect.center = (W // 2, H // 2)
winfont = pygame.font.SysFont('comicsansms', 40)
wintext = winfont.render('You win yay', True, (0, 0, 0))
wintextRect = wintext.get_rect()
wintextRect.center = (W // 2, H // 2)

def detect_collision(dx, dy, ball, rect):
    if dx > 0:
        delta_x = ball.right - rect.left
    else:
        delta_x = rect.right - ball.left
    if dy > 0:
        delta_y = ball.bottom - rect.top
    else:
        delta_y = rect.bottom - ball.top

    if abs(delta_x - delta_y) < 10:
        dx, dy = -dx, -dy
    elif delta_x > delta_y:
        dy = -dy
    else:
        dx = -dx
    return dx, dy

def draw_button(screen, text, position, size, color=(0, 255, 0), text_color=(255, 255, 255)):
    pygame.draw.rect(screen, color, (*position, *size))
    font = pygame.font.SysFont('comicsansms', 30)
    text_surf = font.render(text, True, text_color)
    text_rect = text_surf.get_rect(center=(position[0] + size[0] / 2, position[1] + size[1] / 2))
    screen.blit(text_surf, text_rect)

def is_button_clicked(mouse_pos, position, size):
    x, y = mouse_pos
    px, py = position
    sx, sy = size
    return px <= x <= px + sx and py <= y <= py + sy

settings_button_pos = (W // 2 - 60, H // 2 + 50)
settings_button_size = (120, 40)

while not done:
    current_time = pygame.time.get_ticks()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        elif event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
            is_paused = not is_paused
            if not is_paused:  # Exiting pause resets settings mode
                in_settings = False
            continue
        elif event.type == pygame.MOUSEBUTTONDOWN and is_paused:
            if is_button_clicked(pygame.mouse.get_pos(), settings_button_pos, settings_button_size):
                print("Settings button clicked")
                in_settings = True
        elif event.type == pygame.KEYDOWN and in_settings:
            if event.key == pygame.K_UP:
                ballRadius += 1
                print(f"Ball radius increased to {ballRadius}")
            elif event.key == pygame.K_DOWN:
                ballRadius = max(1, ballRadius - 1)
                print(f"Ball radius decreased to {ballRadius}")
            elif event.key == pygame.K_RETURN:
                in_settings = False
                print("Exiting settings mode")

    if not is_paused:
        current_time = pygame.time.get_ticks()
        screen.fill(bg)
        for color, block in enumerate(block_list):
            if color == bonus_block_index:
                pygame.draw.rect(screen, (255, 215, 0), block)
            else:
                pygame.draw.rect(screen, color_list[color], block)
        for block in unbreakable_block_list:
            pygame.draw.rect(screen, unbreakable_color, block)
        pygame.draw.rect(screen, pygame.Color(255, 255, 255), paddle)
        pygame.draw.circle(screen, pygame.Color(255, 0, 0), ball.center, ballRadius)
        if current_time - last_speed_increase > speed_increase_interval:
            ballSpeed += 1
            paddleW = max(paddleW - 10, 50)
            paddle.x += 5 
            last_speed_increase = current_time
        ball.x += ballSpeed * dx
        ball.y += ballSpeed * dy
        paddle.width = paddleW
        paddle.x = max(min(paddle.x, W - paddleW), 0)
        unbreakable_collision_index = ball.collidelist(unbreakable_block_list)
        if unbreakable_collision_index != -1:
            dx, dy = detect_collision(dx, dy, ball, unbreakable_block_list[unbreakable_collision_index])
        if ball.centerx < ballRadius or ball.centerx > W - ballRadius:
            dx = -dx
        if ball.centery < ballRadius + 50:
            dy = -dy
        if ball.colliderect(paddle) and dy > 0:
            dx, dy = detect_collision(dx, dy, ball, paddle)
        hitIndex = ball.collidelist(block_list)
        if hitIndex != -1:
            if hitIndex == bonus_block_index:  
                paddleW += 50  
            hitRect = block_list.pop(hitIndex)
            color_list.pop(hitIndex)
            dx, dy = detect_collision(dx, dy, ball, hitRect)
            game_score += 1
            collision_sound.play()
        if ball.collidelist(unbreakable_block_list) != -1:
            dx, dy = -dx, -dy
        game_score_text = game_score_fonts.render(f'Your game score is: {game_score}', True, (255, 255, 255))
        screen.blit(game_score_text, game_score_rect)
        if ball.bottom > H:
            screen.fill((0, 0, 0))
            screen.blit(losetext, losetextRect)
        elif not len(block_list):
            screen.fill((255, 255, 255))
            screen.blit(wintext, wintextRect)
        key = pygame.key.get_pressed()
        if key[pygame.K_LEFT] and paddle.left > 0:
            paddle.left -= paddleSpeed
        if key[pygame.K_RIGHT] and paddle.right < W:
            paddle.right += paddleSpeed
    else:
        game_pause_text = game_pause_fonts.render('Game paused. Press ESC to continue.', True, (255, 255, 255))
        game_pause_rect = game_pause_text.get_rect(center=(W // 2, H // 2))
        screen.fill(bg)
        screen.blit(game_pause_text, game_pause_rect)
        if not in_settings:
            draw_button(screen, "Settings", settings_button_pos, settings_button_size)
        pygame.display.flip()

    pygame.display.flip()
    clock.tick(FPS)

pygame.quit()